# 1. Import Libraries 

In [56]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
import os 

In [3]:
os.getcwd()

'C:\\Users\\kames\\OneDrive\\Desktop\\Semester - 4\\Machine Learning\\Project\\notebooks'

In [4]:
train_csv_path = r'..\data\train_test_csv\train_data.csv'
test_csv_path = r'..\data\train_test_csv\test_data.csv'

train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

# Display the number of imported images
print(f"Number of training images: {len(train_df)}")
print(f"Number of test images: {len(test_df)}")

Number of training images: 3365
Number of test images: 374


In [5]:
train_df.Label.value_counts()

Label
airplanes            720
Motorbikes           718
BACKGROUND_Google    420
Faces_easy           392
Faces                391
watch                215
Leopards             180
bonsai               115
car_side             111
ketch                103
Name: count, dtype: int64

In [6]:
test_df.Label.value_counts()

Label
Motorbikes           80
airplanes            80
BACKGROUND_Google    47
Faces                44
Faces_easy           43
watch                24
Leopards             20
bonsai               13
car_side             12
ketch                11
Name: count, dtype: int64

In [7]:
train_df.head()

,HOG_0,HOG_1,HOG_2,HOG_3,HOG_4,HOG_5,HOG_6,HOG_7,HOG_8,HOG_9,...,HOG_8091,HOG_8092,HOG_8093,HOG_8094,HOG_8095,HOG_8096,HOG_8097,HOG_8098,HOG_8099,Label
0,0.000084,5.695531e-05,8.398800e-05,0.000107,0.000327,0.000327,0.000053,0.000070,0.000090,0.000108,...,0.000102,0.000062,3.137357e-05,7.182029e-05,0.000145,1.034424e-04,1.219835e-04,0.000059,0.000134,Faces_easy
1,0.000186,1.357362e-04,1.159534e-04,0.000185,0.000119,0.000049,0.000129,0.000141,0.000151,0.000212,...,0.000348,0.000296,3.011146e-06,0.000000e+00,0.000005,2.000781e-06,7.093396e-06,0.000018,0.000148,watch
2,0.000128,4.212162e-05,6.854870e-05,0.000106,0.000151,0.000131,0.000171,0.000094,0.000131,0.000088,...,0.000089,0.000127,1.672243e-04,9.401813e-05,0.000218,2.107743e-04,2.183743e-04,0.000214,0.000084,car_side
3,0.000329,5.732011e-07,2.427512e-06,0.000002,0.000005,0.000003,0.000003,0.000003,0.000329,0.000329,...,0.000302,0.000000,1.171565e-08,6.417172e-07,0.000006,6.768597e-07,5.347204e-07,0.000001,0.000319,airplanes
4,0.000003,1.289011e-07,3.857198e-07,0.000000,0.000003,0.000000,0.000000,0.000000,0.000003,0.000015,...,0.000021,0.000006,2.203605e-05,2.015302e-06,0.000019,3.331167e-06,1.470229e-05,0.000007,0.000022,airplanes


In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_df['Label'] = label_encoder.fit_transform(train_df['Label'])
test_df['Label'] = label_encoder.transform(test_df['Label'])

In [9]:
test_df.isnull().sum().sum()

0

# Creating Unlabelled Dataset 

In [10]:
from sklearn.model_selection import train_test_split

def partition_and_make_unlabeled(df, test_size=0.5, random_state=None):
    labeled_df1, labeled_df2 = train_test_split(df, test_size=test_size, stratify=df['Label'], random_state=random_state)
    unlabeled_df = labeled_df2.drop('Label', axis=1)
    
    return labeled_df1, labeled_df2, unlabeled_df

In [11]:
labeled_df1, labeled_df2, unlabeled_df = partition_and_make_unlabeled(train_df, test_size=0.5, random_state=42)

# Train Model

In [59]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def trainRF(X, n_estimators=10, max_depth=3, class_weight='balanced', criterion='gini'):
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, class_weight=class_weight, criterion=criterion)
    model.fit(X[:, :-1], X[:, -1])  # Assuming last column is the target label
    return model

def oobe(F, X):
    y = F.predict(X[:, :-1])
    oobe = 1 - np.mean(y == X[:, -1])
    return oobe

def compute_p_star(Xu, Xl):
    num_classes = len(np.unique(Xl[:, -1]))
    return np.random.rand(len(Xu), num_classes)

def draw_random_label(p_star):
    if isinstance(p_star, list):
        p_star = np.array(p_star)
    return np.random.choice(np.arange(len(p_star)))



def stopping_condition(emF, e0F, m, max_epochs):
    if emF > e0F:
        return True
    if abs(emF - e0F) < 0.005:
        return True
    if m >= max_epochs:
        return True
    return False

def semi_supervised_random_forests(Xl, Xu, N=10, T0=1, cooling_function=cooling_function, stopping_condition=stopping_condition, max_epochs=100, n_estimators=10, max_depth=3, class_weight='balanced', criterion='gini'):
    F = trainRF(Xl, n_estimators=n_estimators, max_depth=max_depth, class_weight=class_weight, criterion=criterion)
    e0F = oobe(F, Xl)
    emF = 0
    m = 0

    while not stopping_condition(emF, e0F, m, max_epochs):
        Tm = cooling_function(T0, m)
        m += 1
        p_star = compute_p_star(Xu, Xl)

        for _ in range(N):
            y_hat_u = [draw_random_label(p_star[i]) for i in range(len(p_star))]
            Xn = np.vstack((Xl, np.column_stack((Xu, y_hat_u))))
            F.estimators_[_] = trainRF(Xn, n_estimators=len(F.estimators_), max_depth=max_depth, class_weight=class_weight, criterion=criterion)

        emF = oobe(F, Xl)

        if emF > e0F:
            F = trainRF(Xl, n_estimators=n_estimators, max_depth=max_depth, class_weight=class_weight, criterion=criterion)
    return Xn


In [60]:
trained_forest= semi_supervised_random_forests(labeled_df1.values, unlabeled_df.values, N=10, T0=1, cooling_function=cooling_function, stopping_condition=stopping_condition, max_epochs=100)

AttributeError: 'RandomForestClassifier' object has no attribute '_support_missing_values'

In [34]:
labeled_df1.head()

,HOG_0,HOG_1,HOG_2,HOG_3,HOG_4,HOG_5,HOG_6,HOG_7,HOG_8,HOG_9,...,HOG_8091,HOG_8092,HOG_8093,HOG_8094,HOG_8095,HOG_8096,HOG_8097,HOG_8098,HOG_8099,Label
2868,0.000048,0.000021,0.000025,0.000310,0.000310,1.337568e-04,2.724245e-05,0.000009,0.000041,0.000207,...,0.000204,0.000204,0.000204,0.000113,0.000124,0.000084,0.000083,0.000138,0.000204,2
130,0.000312,0.000008,0.000009,0.000012,0.000009,3.597714e-06,4.095231e-06,0.000011,0.000312,0.000312,...,0.000314,0.000134,0.000070,0.000073,0.000127,0.000059,0.000063,0.000182,0.000314,1
2380,0.000063,0.000207,0.000259,0.000259,0.000192,2.507228e-05,1.410853e-05,0.000015,0.000047,0.000075,...,0.000075,0.000062,0.000185,0.000245,0.000245,0.000178,0.000058,0.000020,0.000025,5
2098,0.000154,0.000146,0.000165,0.000083,0.000178,1.199484e-04,6.326045e-05,0.000105,0.000149,0.000186,...,0.000073,0.000054,0.000107,0.000076,0.000224,0.000037,0.000054,0.000053,0.000084,9
796,0.000439,0.000003,0.000002,0.000000,0.000003,5.867088e-07,4.022561e-07,0.000003,0.000439,0.000439,...,0.000271,0.000009,0.000026,0.000000,0.000047,0.000000,0.000074,0.000025,0.000271,5


In [36]:
model = RandomForestClassifier(n_estimators = 100, max_depth = 3, class_weight='balanced', criterion = 'gini')

In [41]:
model.fit(xn[:, :-1], xn[:, -1])

RandomForestClassifier(class_weight='balanced', max_depth=3)

In [ ]:
trained_forest = semi_supervised_random_forests(labeled_df1.values, unlabeled_df.values, N=5, T0=1, cooling_function=cooling_function, stopping_condition=stopping_condition, max_epochs=1000)

In [ ]:
y_pred = trained_forest.predict(test_df.values[:,:-1])

In [ ]:
y_true  = test_df.values[:,-1]

In [ ]:
y_true = y_true.astype(int)
y_pred = y_pred.astype(int)

In [ ]:
y_true

In [ ]:
import numpy as np

def precision_recall(y_true, y_pred):
  """Calculates precision and recall for multiclass classification.

  Args:
    y_true: A numpy array of the true labels.
    y_pred: A numpy array of the predicted labels.

  Returns:
    A tuple of (precision, recall).
  """

  # Calculate the confusion matrix.
  confusion_matrix = np.zeros((y_true.max() + 1, y_true.max() + 1))
  for i in range(len(y_true)):
    confusion_matrix[y_true[i], y_pred[i]] += 1

  # Calculate precision and recall for each class.
  precision = []
  recall = []
  for i in range(y_true.max() + 1):
    tp = confusion_matrix[i, i]
    fp = np.sum(confusion_matrix[:, i]) - tp
    fn = np.sum(confusion_matrix[i, :]) - tp
    precision.append(tp / (tp + fp))
    recall.append(tp / (tp + fn))

  # Calculate the overall precision and recall.
  precision = np.mean(precision)
  recall = np.mean(recall)

  return precision, recall

precision, recall = precision_recall(y_true, y_pred)

print("Precision:", precision)
print("Recall:", recall)